In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [39]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()

        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.linear1 = nn.Linear(16*6*6,120)
        self.linear2 = nn.Linear(120,84)
        self.linear3 = nn.Linear(84,10)

    def forward(self,image):
        image = self.conv1(image).relu()
        image = F.max_pool2d(image,(2,2))
        image = self.conv2(image).relu()
        image = F.max_pool2d(image,(2,2))

        image = image.view(-1, self.num_flat_features(image))
        image = F.relu(self.linear1(image))
        image = F.relu(self.linear2(image))
        image = self.linear3(image)
        return F.softmax(image)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [40]:
#### Create LeNet Model 
model = LeNet()

In [41]:
### Show the model architecture
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (linear1): Linear(in_features=576, out_features=120, bias=True)
  (linear2): Linear(in_features=120, out_features=84, bias=True)
  (linear3): Linear(in_features=84, out_features=10, bias=True)
)


In [44]:
### Passing a image  of random 32*32 pixels
image = torch.rand((1,1,32,32))
print(image.shape)

print("Output: ",model(image))

torch.Size([1, 1, 32, 32])
Output:  tensor([[0.1072, 0.1023, 0.0948, 0.1051, 0.0935, 0.0993, 0.1098, 0.0959, 0.0930,
         0.0992]], grad_fn=<SoftmaxBackward0>)


/var/folders/gq/c7ch578s2k7_h10mbq482nb00000gp/T/ipykernel_5412/122311029.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(image)


In [54]:
####### Load cifar10 data

%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms


transforms = transforms.Compose([transforms.ToTensor(),
transforms.Grayscale(),transforms.Normalize((0.5), (0.5))])



trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transforms)



Files already downloaded and verified


In [55]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True)

In [59]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

In [62]:
for epoch in range(10):
    running_loss = 0.0
    for i,train_batch in enumerate(trainloader,0):
        images, labels = train_batch
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

/var/folders/gq/c7ch578s2k7_h10mbq482nb00000gp/T/ipykernel_5412/122311029.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(image)


[1,  2000] loss: 2.306
[1,  4000] loss: 2.303
[1,  6000] loss: 2.303
[1,  8000] loss: 2.303
[1, 10000] loss: 2.303
[1, 12000] loss: 2.303
[2,  2000] loss: 2.303
[2,  4000] loss: 2.303
[2,  6000] loss: 2.303
[2,  8000] loss: 2.303
[2, 10000] loss: 2.303
[2, 12000] loss: 2.303
[3,  2000] loss: 2.303
[3,  4000] loss: 2.303
[3,  6000] loss: 2.303
[3,  8000] loss: 2.303
[3, 10000] loss: 2.303
[3, 12000] loss: 2.303
[4,  2000] loss: 2.303
[4,  4000] loss: 2.303
[4,  6000] loss: 2.303
[4,  8000] loss: 2.303
[4, 10000] loss: 2.303
[4, 12000] loss: 2.303
[5,  2000] loss: 2.303


KeyboardInterrupt: 